# Fazendo as inferências nas imagens

In [1]:
# Tentar rodar em paralelo - OK
# Abrir a imagem - OK
# gerando GrayScale - OK
# aplicar Scaler - OK
# gerar Filtro de Sobel - OK
# aplicar Scaler ao Sobel - OK
# gerar LBP - OK
# aplicar Scaler no LBP - OK
# separar as quadriculas
# aplicar PCA
# Inferir se é faixa de pedestre    
# salvar imagem 1bit por SCM

## DEpois gerando dados

# agrupar em uma única imagem para toda a ciade
# agrupar dbScan
# gerar polígonos
# gerar linhas

In [52]:
import multiprocessing as mp
import geopandas as gpd
import rasterio
from rasterio import features
import numpy as np
import glob
import joblib
from skimage import filters
from skimage.feature import local_binary_pattern
from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [3]:
gdf_scms = gpd.read_file('downloads/SIRGAS_SHP_quadriculaortofoto2017.zip!SIRGAS_SHP_quadriculaortofoto2017/SIRGAS_SHP_quadriculaortofoto2017.shp')

In [4]:
n_cpu = mp.cpu_count() - 1
# pool = mp.Pool(processes=n_cpu)


In [17]:
sample =  gdf_scms[2:17]['qo2010_cod'].to_list()

In [43]:
len(sample)

15

In [51]:
grayscale_scaler = joblib.load('pre-process/grayscale.scaler.save')
sobel_scaler = joblib.load('pre-process/sobel.scaler.save') 
lbp_scaler = joblib.load('pre-process/lbp.scaler.save')

In [53]:
radius = 3
n_points = 8 * radius
tamanho_janela = 24 # tamanho em pixels
sobreposicao_de_janelas = 12 # sobreposição das janelas em pixels

In [54]:
def faixas(scm):
    file = glob.glob(f'../ortofotos-2017/RGB-2017/*{scm}*.jp2')
    dataset = rasterio.open(file[0])

    grayscale = 0.2125 * dataset.read(1) + \
            0.7154 * dataset.read(2) + \
            0.0721 * dataset.read(3)
    grayscale = grayscale.astype('uint8')

    grayscale = grayscale_scaler.transform(grayscale.reshape(-1, 1)).reshape(grayscale.shape)

    sobel = filters.sobel(grayscale)
    sobel = sobel_scaler.transform(sobel.reshape(-1, 1)).reshape(sobel.shape)

    lbp = local_binary_pattern(grayscale, n_points, radius)
    lbp = lbp_scaler.transform(lbp.reshape(-1, 1)).reshape(lbp.shape)
    
    return dataset.width, dataset.height

In [55]:
with mp.Pool(processes=n_cpu) as p:
    p.map(faixas, sample)

In [39]:
def f(x):
    return x*x

with mp.Pool(5) as p:
    print(p.map(f, np.arange(100000)))